In [1]:
# Tải thư viện cần thiết
import numpy as np
import pandas as pd


# Đáp án đúng cho bài thi (25 câu hỏi)
answer_key = [
    "B", "A", "D", "D", "C", "B", "D", "A", "C", "C",
    "D", "B", "A", "B", "A", "C", "B", "D", "A", "C",
    "A", "A", "B", "D", "D"
]


def get_filename():
    """Nhập tên file và kiểm tra xem file có tồn tại không.
    Returns:
        str: Tên file hợp lệ (bao gồm đuôi .txt).
    """
    while True:
        # Nhập tên file từ người dùng và thêm đuôi .txt
        filename = input("Enter a class file to grade (i.e. class1 for class1.txt): ") + ".txt"
        try:
            # Thử mở file để kiểm tra xem file có tồn tại không
            with open(filename, "r") as file:
                print(f"Successfully opened {filename}")
                return filename
        except FileNotFoundError:
            # Nếu file không tồn tại, thông báo lỗi và yêu cầu nhập lại
            print("File cannot be found.")
            continue


def process_data(filename):
    """Xử lý dữ liệu: kiểm tra tính hợp lệ, chấm điểm.

    Args:
        filename (str): Tên file dữ liệu cần xử lý.

    Returns:
        tuple: (valid_data, invalid_data, scores, question_skips, question_wrongs)
            - valid_data: Danh sách dữ liệu hợp lệ (ID, điểm).
            - invalid_data: Danh sách dòng không hợp lệ.
            - scores: Danh sách điểm số của học sinh.
            - question_skips: Mảng đếm số lần bỏ qua cho từng câu hỏi.
            - question_wrongs: Mảng đếm số lần trả lời sai cho từng câu hỏi.
    """
    # Thử mở file để đọc dữ liệu
    try:
        with open(filename, "r") as file:
            lines = file.readlines()
    except FileNotFoundError:
        # Nếu file không tồn tại, trả về None cho tất cả giá trị
        print(f"Error: Could not open {filename}")
        return None, None, None, None, None

    # Khởi tạo danh sách để lưu dữ liệu hợp lệ, không hợp lệ và điểm số
    valid_data = []
    invalid_data = []
    scores = []
    # Khởi tạo mảng để đếm số lần bỏ qua và trả lời sai cho từng câu hỏi
    question_skips = np.zeros(25)
    question_wrongs = np.zeros(25)

    # In tiêu đề phân tích
    print("\n**** ANALYZING ****")
    # Duyệt qua từng dòng trong file
    for line in lines:
        # Tách dòng thành các phần tử bằng dấu phẩy
        parts = line.strip().split(",")
        # Kiểm tra số lượng giá trị (phải có 26 giá trị: 1 ID + 25 câu trả lời)
        if len(parts) != 26:
            print(f"Invalid line of data: does not contain exactly 26 values:\n{line.strip()}")
            invalid_data.append(line.strip())
            continue

        # Lấy ID và câu trả lời của học sinh
        student_id, answers = parts[0], parts[1:]
        # Kiểm tra định dạng ID: bắt đầu bằng "N", dài 9 ký tự, 8 ký tự sau là số
        if not (student_id.startswith("N") and len(student_id) == 9 and student_id[1:].isdigit()):
            print(f"Invalid line of data: N# is invalid\n{line.strip()}")
            invalid_data.append(line.strip())
            continue

        # Chấm điểm cho học sinh
        score = 0
        for i in range(25):
            # Nếu câu trả lời trống, tăng số lần bỏ qua
            if answers[i] == "":
                question_skips[i] += 1
            # Nếu câu trả lời đúng, cộng 4 điểm
            elif answers[i] == answer_key[i]:
                score += 4
            # Nếu câu trả lời sai, trừ 1 điểm và tăng số lần sai
            else:
                score -= 1
                question_wrongs[i] += 1

        # Lưu dữ liệu hợp lệ và điểm số
        valid_data.append((student_id, score))
        scores.append(score)

    # Nếu không có dòng không hợp lệ, thông báo không có lỗi
    if not invalid_data:
        print("No errors found!")
    return valid_data, invalid_data, scores, question_skips, question_wrongs


def calculate_statistics(scores, question_skips, question_wrongs):
    """Tính toán thống kê từ danh sách điểm số và câu hỏi.

    Args:
        scores (list): Danh sách điểm số của học sinh.
        question_skips (np.ndarray): Mảng đếm số lần bỏ qua cho từng câu hỏi.
        question_wrongs (np.ndarray): Mảng đếm số lần trả lời sai cho từng câu hỏi.

    Returns:
        dict: Từ điển chứa các thống kê (điểm trung bình, trung vị, v.v.).
    """
    # Nếu không có điểm số, trả về None
    if not scores:
        return None

    # Sử dụng pandas để lưu trữ và tính toán thống kê
    df = pd.DataFrame(scores, columns=["score"])
    total_students = len(scores)

    # Tính số học sinh có điểm cao (>80)
    high_scores = len(df[df["score"] > 80])
    # Tính điểm trung bình
    mean_score = df["score"].mean()
    # Tính điểm cao nhất
    highest_score = df["score"].max()
    # Tính điểm thấp nhất
    lowest_score = df["score"].min()
    # Tính miền giá trị (cao nhất - thấp nhất)
    range_scores = highest_score - lowest_score
    # Tính giá trị trung vị
    median_score = df["score"].median()

    # Tìm câu hỏi bị bỏ qua nhiều nhất
    max_skips = np.max(question_skips)
    # Lấy danh sách câu hỏi có số lần bỏ qua cao nhất
    skip_info = [
        f"{i+1} - {int(question_skips[i])} - {question_skips[i]/total_students:.3f}"
        for i in range(25)
        if question_skips[i] == max_skips
    ]

    # Tìm câu hỏi bị trả lời sai nhiều nhất
    max_wrongs = np.max(question_wrongs)
    # Lấy danh sách câu hỏi có số lần sai cao nhất
    wrong_info = [
        f"{i+1} - {int(question_wrongs[i])} - {question_wrongs[i]/total_students:.3f}"
        for i in range(25)
        if question_wrongs[i] == max_wrongs
    ]

    # Trả về từ điển chứa các thống kê
    return {
        "high_scores": high_scores,
        "mean_score": mean_score,
        "highest_score": highest_score,
        "lowest_score": lowest_score,
        "range_scores": range_scores,
        "median_score": median_score,
        "skip_info": skip_info,
        "wrong_info": wrong_info,
        "total_students": total_students,
    }


def save_results(filename, valid_data):
    """Lưu kết quả vào file.

    Args:
        filename (str): Tên file dữ liệu gốc.
        valid_data (list): Danh sách dữ liệu hợp lệ (ID, điểm).
    """
    # Tạo tên file kết quả bằng cách thay .txt thành _grades.txt
    output_filename = filename.replace(".txt", "_grades.txt")
    # Mở file để ghi kết quả
    with open(output_filename, "w") as file:
        # Ghi từng dòng: ID, điểm
        for student_id, score in valid_data:
            file.write(f"{student_id},{score}\n")
    # Thông báo đã lưu file thành công
    print(f"Results saved to {output_filename}")


def main():
    """Chương trình chính: điều phối các bước từ nhập file đến lưu kết quả."""
    # Bước 1: Nhập tên file và kiểm tra tính hợp lệ
    filename = get_filename()

    # Bước 2: Xử lý dữ liệu và chấm điểm
    valid_data, invalid_data, scores, question_skips, question_wrongs = process_data(filename)

    # Nếu không mở được file, thoát chương trình
    if valid_data is None:
        return

    # Bước 3: In báo cáo số dòng hợp lệ và không hợp lệ
    print("\n**** REPORT ****")
    print(f"Total valid lines of data: {len(valid_data)}")
    print(f"Total invalid lines of data: {len(invalid_data)}")

    # Nếu không có dữ liệu hợp lệ, thoát chương trình
    if not valid_data:
        return

    # Bước 4: Tính toán thống kê
    stats = calculate_statistics(scores, question_skips, question_wrongs)

    # In các thống kê
    print(f"Total student of high scores: {stats['high_scores']}")
    print(f"Mean (average) score: {stats['mean_score']:.3f}")
    print(f"Highest score: {stats['highest_score']}")
    print(f"Lowest score: {stats['lowest_score']}")
    print(f"Range of scores: {stats['range_scores']}")
    print(f"Median score: {stats['median_score']:.3f}")
    print(f"Question that most people skip: {', '.join(stats['skip_info'])}")
    print(f"Question that most people answer incorrectly: {', '.join(stats['wrong_info'])}")

    # Bước 5: Lưu kết quả vào file
    save_results(filename, valid_data)


if __name__ == "__main__":
    main()


Enter a class file to grade (i.e. class1 for class1.txt):  class1


Successfully opened class1.txt

**** ANALYZING ****
No errors found!

**** REPORT ****
Total valid lines of data: 20
Total invalid lines of data: 0
Total student of high scores: 6
Mean (average) score: 75.600
Highest score: 91
Lowest score: 59
Range of scores: 32
Median score: 73.000
Question that most people skip: 3 - 4 - 0.200, 5 - 4 - 0.200, 23 - 4 - 0.200
Question that most people answer incorrectly: 10 - 4 - 0.200, 14 - 4 - 0.200, 16 - 4 - 0.200, 19 - 4 - 0.200, 22 - 4 - 0.200
Results saved to class1_grades.txt
